<br><br>
<center><font size='5'><b>Deep Learning for All_pytorch</b></font><br><br><font size='5'>CNN_MNIST_Visdom_Losstracking<b></b></font>

# Convolution

![image](https://user-images.githubusercontent.com/48466625/61202926-e76ec100-a723-11e9-92ad-2579a9c28b4e.png)

## nn.Conv2d, MaxPool2d

- torch.nn.Conv2d(input_channels, output_channels, kernel_size, strice=1, padding=0, dilation=1, groups=1, bias=True)
  - padding=0 : 패딩없이
  - torch.nn.Conv2d(1,1,11) : 채널 한개짜리의 11x11 커널 
- Input 
  - type : torch.Tensor
  - input shape : (배치사이즈, 채널, height, width)
  
- torch.nn.MaxPool2d(kernel_size, stride=None, paddig=0, dilation=1, return_indices=False, ceil_mode=False)
  
![image](https://user-images.githubusercontent.com/48466625/61202607-f012c780-a722-11e9-9eda-ad81bd417939.png)


In [1]:
import torch
import torch.nn as nn

inputs = torch.Tensor(1,1,28,28)

In [3]:
conv1 = nn.Conv2d(1,5,5) # 인풋 채널은 1개, 아웃풋 채널은 5개, 5*5 사이즈

In [4]:
pool = nn.MaxPool2d(2)

In [5]:
out1 = conv1(inputs)
out2 = pool(out1)

In [6]:
out1.size()

torch.Size([1, 5, 24, 24])

In [7]:
out2.size()

torch.Size([1, 5, 12, 12])

# CNN with MNIST

![image](https://user-images.githubusercontent.com/48466625/61203361-1e91a200-a725-11e9-92e1-1b3000ab9a32.png)


In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

In [12]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [3]:
print(device)

cpu


In [13]:
# parameters
learning_rate = 0.001
training_epochs = 5
batch_size = 100

In [5]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [6]:
# Data Loader

data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         drop_last=True)

In [14]:
# CNN Model (2 conv layers)
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # Final FC 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.fc(out)
        return out

In [15]:
# instantiate CNN model
model = CNN().to(device)

In [16]:
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)

In [18]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# train my model
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

Learning started. It takes sometime.


# Visdom

- python -m visdom.server
- https://github.com/facebookresearch/visdom

In [1]:
import torch
import torch.nn as nn

import torchvision
import torchvision.datasets as dsets

In [2]:
import visdom
vis = visdom.Visdom()

## Text

In [3]:
vis.text("Hello, world!", env='main') # 나중에 끌때 main에 있는 것 들은 다 끈다고 할때 사용

'window_378913395ed1f8'

## Image / Images

In [4]:
vis.image(torch.randn(3,200,200))

'window_37891356630254'

In [8]:
vis.images(torch.Tensor(3,3,28,28))

'window_3789137a67a8e4'

## MNIST and CIFAR10

In [9]:
MNIST = dsets.MNIST(root="./MNIST_data", train=True, transform=torchvision.transforms.ToTensor(), download=True)
cifar10 = dsets.CIFAR10(root="./cifar10", train=True, transform=torchvision.transforms.ToTensor(), download=True)

100.0%

In [12]:
data = cifar10.__getitem__(10) # 10번째 데이터 가져와서
print(data[0].shape)
vis.images(data[0], env='main')

torch.Size([3, 32, 32])


'window_3789143c71d71c'

In [11]:
data = MNIST.__getitem__(0)
print(data[0].shape)
vis.images(data[0], env='main')

torch.Size([1, 28, 28])


'window_37891439fae6d6'

In [16]:
vis.close(env='main') # visdom에 뜬 창들 다 꺼짐

''

### data_loader를 통해서 여러개 한번에

In [13]:
data_loader = torch.utils.data.DataLoader(dataset=MNIST,
                                         batch_size=32,
                                         shuffle=False)

In [14]:
for num, value in enumerate(data_loader):
    value = value[0]
    print(value.shape)
    vis.images(value)
    break

torch.Size([32, 1, 28, 28])


## Line Plot

- 학습이 진행되면서 경과를 그래프로 보고싶을 때

In [20]:
Y_data = torch.randn(5)
plt = vis.line(Y=Y_data)

In [21]:
X_data = torch.Tensor([1,2,3,4,5])
plt = vis.line(Y=Y_data, X=X_data)

### Line update

In [22]:
Y_append = torch.randn(1)
X_append = torch.Tensor([6])

vis.line(Y=Y_append, X=X_append, win=plt, update='append')

'window_378914e39296aa'

### Multiple lines

In [25]:
num = torch.Tensor(list(range(0,10)))
print(num.shape)
num = num.view(-1,1)
print(num.shape)
num = torch.cat((num,num),dim=1)
print(num.shape)
#### x값들 동일한 리스트 2개 만들어 주려고 ####

plt = vis.line(Y=torch.randn(10,2), X=num)

torch.Size([10])
torch.Size([10, 1])
torch.Size([10, 2])


### Line information

In [26]:
plt = vis.line(Y=Y_data, X=X_data, opts=dict(title='Test', legend=['범주1'], showlegend=True) )

In [27]:
plt = vis.line(Y=torch.randn(10,2), X=num, opts=dict(title='Test', legend=['1번', '2번'], showlegend=True))

### Makfe function for update line

In [28]:
def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value는 텐서값'''
    
    vis.line(X=num, Y=loss_value, win=loss_plot, update='append')

In [30]:
plt = vis.line(Y=torch.Tensor(1).zero_())

for i in range(500):
    loss = torch.randn(1) + i
    loss_tracker(plt, loss, torch.Tensor([i]))

In [31]:
vis.close(env='main')

''

# MNIST + visdom 적용

## import 

In [32]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init
import visdom
vis = visdom.Visdom()

## Define loss_tracker

In [33]:
def loss_tracker(loss_plot, loss_value, num):
    
    vis.line(X=num, Y=loss_value, win=loss_plot, update='append')

## Model

In [34]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [35]:
# parameters
learning_rate = 0.001
training_epochs = 5
batch_size = 100

In [36]:
# dataset
mnist_train = dsets.MNIST(root="./MNIST_data", train=True, transform=torchvision.transforms.ToTensor(), download=True)
mnist_test =  dsets.MNIST(root="./MNIST_data", train=False, transform=torchvision.transforms.ToTensor(), download=True)

In [37]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size,
                                          shuffle=True, drop_last=True)

In [38]:
# CNN Model (2 conv layers)
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # Final FC 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.fc(out)
        return out

In [39]:
model = CNN().to(device)

In [41]:
value = (torch.Tensor(1,1,28,28).to(device))
print(model(value).shape)

torch.Size([1, 10])


In [42]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)    # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## Make plot

In [44]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(), opts=dict(title='loss_tracker', legend=['loss'],showlegend=True))

# 0 값을 하나 넣어서 만들 것 

## Train with loss_tracker

In [45]:
# train my model
total_batch = len(data_loader)

print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch]))
print('Learning Finished!')

Learning started. It takes sometime.
[Epoch:    1] cost = 0.223587215
[Epoch:    2] cost = 0.0620054752
[Epoch:    3] cost = 0.0448652655
[Epoch:    4] cost = 0.0355408676
[Epoch:    5] cost = 0.0291333236
Learning Finished!
